In [20]:
%pwd 

'd:\\ML_Deployment\\Automadata'

In [2]:
import os 
os.chdir('../')

In [3]:
%pwd

'd:\\ML_Deployment\\Automadata'

In [4]:
import joblib
import numpy as np 
import pandas as pd 
from pathlib import Path 
from dataclasses import dataclass
from src.logging import logger
from src.constants import *
from src.utils.common import *

In [71]:
PREPROCESSOR_OBJ_PATH = "artifacts/data_transformation/preprocessor.joblib"
MODEL_OBJ_PATH = "artifacts/model_trainer/clf_model.joblib"

In [7]:
from src.components.data_transformation import DataTransformation
from src.config.configuration import ConfigManager
from src.entity.config_entity import DataTransformationConfig

In [88]:
class PredictionPipeline:
    def __init__(self, data_transform_config):
        try:
            self.data_transform = DataTransformation(data_transform_config)
            self.preprocessor = joblib.load(Path(PREPROCESSOR_OBJ_PATH))
            self.model = joblib.load(Path(MODEL_OBJ_PATH))
        except Exception as e:
            raise RuntimeError(f"Error loading preprocessor or model: {e}")

    def model_prediction(self, data):
        """
        Make predictions using the loaded preprocessor and model.

        Args:
            data (pd.DataFrame): Input data for prediction.

        Returns:
            tuple: A tuple containing the predicted labels and predicted probabilities.
        """
        try:
            features = self.data_transform.generate_features(data)
            data_transformed = self.preprocessor.transform(features)
            prediction = self.model.predict(data_transformed)
            predict_prob = self.model.predict_proba(data_transformed)[:,-1]
            return prediction, predict_prob
        except Exception as e:
            raise RuntimeError(f"Error making predictions: {e}")

In [89]:
data_transform_config = DataTransformationConfig

In [90]:
predict_pipeline= PredictionPipeline(data_transform_config)

In [91]:
res, pred_proba = predict_pipeline.model_prediction(test_data)

In [92]:
res, pred_proba

(array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]),
 array([0.65826553, 0.17066029, 0.8160412 , 0.06470333, 0.00482792,
        0.8008575 , 0.6733873 , 0.17748894, 0.09692879, 0.66033685,
        0.71845347, 0.936299  , 0.24474007, 0.2722317 , 0.17777535,
        0.1766214 , 0.06834161, 0.04261632, 0.03448173, 0.30696705,
        0.0709138 , 0.55926657, 0.50420356, 0.31273413, 0.07806255,
        0.51465374, 0.6003771 , 0.6536493 , 0.8718947 , 0.81741893,
        0.90742725, 0.7104256 , 0.8642303 , 0.6717175 , 0.9141675 ,
        0.7203101 , 0.77411675, 0.9354172 , 0.4293785 , 0.81500465,
        0.8010178 , 0.9412297 , 0.7584153 , 0.73955053, 0.9861098 ,
        0.83145124, 0.8416393 , 0.5331895 , 0.95014435, 0.7495083 ],
       dtype=float32))